In [93]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import balanced_accuracy_score

In [94]:
x_train=pd.read_csv("data/artificial_train.data",header=None, sep=" ")

In [95]:
x_train=x_train.drop(columns=[500])

In [96]:
y_train=pd.read_csv("data/artificial_train.labels",header=None, sep=" ")

In [97]:
y_train=y_train.rename(columns = {0:'y'})

In [98]:
pca = PCA(n_components=10)

In [99]:
pca.fit(x_train)

PCA(n_components=10)

In [100]:
x_train=pca.transform(x_train)

In [101]:
x_train.shape

(2000, 10)

In [102]:
reg = LogisticRegression().fit(x_train, y_train.to_numpy().reshape(-1))

pred = reg.predict(x_train)

balanced_accuracy_score(y_train, pred)

In [109]:
forest=RandomForestClassifier().fit(x_train, y_train.to_numpy().reshape(-1))

pred = forest.predict(x_train)

balanced_accuracy_score(y_train, pred)

In [ ]:
sum(pca.explained_variance_ratio_)

In [113]:
x_train[1::4]

array([[ 244.79417885, -157.89263961,  -14.27752663, ...,  -43.12728089,
           4.61581382,   80.15195047],
       [ -78.54720669, -317.84607987,  106.09666065, ...,  -78.16062008,
          10.27588287,  100.69783874],
       [-144.99267469,  127.86061219, -288.71015659, ...,   14.00807946,
         -21.82871079,   -7.76193485],
       ...,
       [-154.62796816,  169.47901309,  132.99864798, ...,   83.45591149,
          20.29488694,    0.86163305],
       [-279.04939447,  258.2374467 ,   32.58506282, ...,   14.15902391,
          24.24657543,  -16.32870432],
       [-376.82786032,   84.81547175,  -97.07825194, ...,  -22.63241917,
         -47.10880295,   20.07366526]])

In [119]:
acc_reg=[0 for x in range(5)]
acc_forest=[0 for x in range(5)]
for i in range(5):
    mask = np.zeros(x_train.shape[0], dtype=bool)
    mask[i::5] = True

    x_test = x_train[mask]
    x_fold = x_train[~mask]
    
    y_test = y_train[mask]
    y_fold = y_train[~mask]
    
    reg = LogisticRegression().fit(x_fold, y_fold.to_numpy().reshape(-1))
    pred = reg.predict(x_test)
    acc_reg[i]=balanced_accuracy_score(y_test, pred)
    
    forest=RandomForestClassifier().fit(x_fold, y_fold.to_numpy().reshape(-1))
    pred = forest.predict(x_test)
    acc_forest[i]=balanced_accuracy_score(y_test, pred)
    

In [121]:
acc_forest

[0.840617869053888,
 0.833024722250025,
 0.8151629072681704,
 0.8073057305730573,
 0.8554924858093071]

In [122]:
acc_reg

array([ 1, -1, -1,  1, -1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1, -1, -1,
       -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1, -1,  1, -1,
        1, -1, -1,  1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,
        1,  1, -1,  1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1,
       -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,  1, -1,  1, -1,  1,  1,
       -1,  1, -1,  1,  1, -1, -1,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1,
        1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1,  1,
        1,  1,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,
        1, -1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1, -1, -1,  1,
       -1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1,  1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1,
        1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1,  1,
        1,  1, -1,  1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1,  1, -1,
       -1, -1,  1,  1, -1